In [1]:
import os
import torch
import numpy as np
from glob import glob
from tqdm.auto import tqdm
from sklearn.linear_model import LogisticRegression
import pandas as pd
from utils.store_data import ReadData
import numpy as np
import torch
from utils.arguments import Arguments
from tqdm import tqdm
from sklearn.decomposition import PCA
from linear_probe import get_linear_acc
import random
import os
from sklearn.metrics import roc_auc_score,accuracy_score,roc_curve

In [2]:
data_reader=ReadData('/root/autodl-fs/hallscope/pope/popular/train')
data=data_reader.read_all()
random.shuffle(data)

In [3]:
df=pd.DataFrame(data)

In [4]:
df.head()

,img_path,question,label,question_id,category,most_likely
0,[/root/autodl-tmp/coco_images/train/COCO_train...,[Is there an airplane in the imange?\nAnswer t...,"[1, 0, 1, 0, 1, 0, 1, 0]","[tensor(15153), tensor(15154), tensor(15155), ...","[popular, popular, popular, popular, popular, ...",{'embedding': [[[ 0.00389099 0.00149536 0.00...
1,[/root/autodl-tmp/coco_images/train/COCO_train...,[Is there a person in the image?\nAnswer the q...,"[1, 0, 1, 0, 1, 0, 1, 0]","[tensor(8361), tensor(8362), tensor(8363), ten...","[popular, popular, popular, popular, popular, ...",{'embedding': [[[ 0.00389099 0.00149536 0.00...
2,[/root/autodl-tmp/coco_images/train/COCO_train...,[Is there a kite in the image?\nAnswer the que...,"[1, 0, 1, 0, 1, 0, 1, 0]","[tensor(17583), tensor(17584), tensor(17585), ...","[popular, popular, popular, popular, popular, ...",{'embedding': [[[ 0.00389099 0.00149536 0.00...
3,[/root/autodl-tmp/coco_images/train/COCO_train...,[Is there a skateboard in the image?\nAnswer t...,"[1, 0, 1, 0, 1, 0, 1, 0]","[tensor(6857), tensor(6858), tensor(6859), ten...","[popular, popular, popular, popular, popular, ...",{'embedding': [[[ 0.00389099 0.00149536 0.00...
4,[/root/autodl-tmp/coco_images/train/COCO_train...,[Is there a chair in the image?\nAnswer the qu...,"[1, 0, 1, 0, 1, 0, 1, 0]","[tensor(8129), tensor(8130), tensor(8131), ten...","[popular, popular, popular, popular, popular, ...",{'embedding': [[[ 0.00389099 0.00149536 0.00...


In [5]:
embeddings=df['most_likely'].apply(lambda x: x['embedding']).to_list()

In [6]:
embeddings=np.concatenate(embeddings,axis=0)

In [7]:
embeddings.shape

(20000, 33, 4096)

In [8]:
gt_label=df['label'].to_list()
gt_label=np.concatenate(gt_label,axis=0)

In [9]:
gt_label.shape

(20000,)

In [10]:
responses=df['most_likely'].apply(lambda x: x['response']).to_list()
resps=[]
for x in responses:
    resps+=x

In [11]:
is_hall=[]
for i,x in enumerate(resps):
    if x.lower()=='no' and gt_label[i]==0:
        is_hall.append(1)
    elif x.lower()=='yes' and gt_label[i]==1:
        is_hall.append(1)
    else:
        is_hall.append(0)
is_hall=np.array(is_hall)

In [12]:
is_hall.shape

(20000,)

In [13]:
print(f'There are {len(is_hall)} responses. {sum(is_hall==1)} answer correctly and {sum(is_hall==0)} answer wrongly')

There are 20000 responses. 17437 answer correctly and 2563 answer wrongly


In [14]:
length=len(is_hall)
train_rate=0.8

all_indices = np.random.permutation(length)
train_val_idxs = all_indices[:int(
    train_rate * length)]  # trainset and validation index

test_idxs=all_indices[int(
    train_rate * length):]  #test index

validset_len = 1000
# exclude validation samples.
train_idxs = train_val_idxs[:len(
    train_val_idxs) - validset_len]  # trainset index
val_idxs = train_val_idxs[len(
    train_val_idxs) - validset_len:]  # validation index
hall_label_test = []
hall_label_wild = []
hall_label_val = []

'''get testset, wildset and valset. The valset is used for determining the hype-parameters'''
hall_label_test = is_hall[test_idxs]
hall_label_wild = is_hall[train_idxs]
hall_label_val = is_hall[val_idxs]

print(f'trainset length:{len(hall_label_wild)}')
print(f'validset length:{len(hall_label_val)}')
print(f'testset length:{len(hall_label_test)}')
print(f'trainset postive: {sum(hall_label_wild==1)} negative: {sum(hall_label_wild==0)}')
print(
    f'testset postive: {sum(hall_label_test==1)} negative: {sum(hall_label_test==0)}')

trainset length:15000
validset length:1000
testset length:4000
trainset postive: 13067 negative: 1933
testset postive: 3503 negative: 497


In [15]:
all_embeddings = embeddings
all_embeddings.shape

(20000, 33, 4096)

In [16]:
embedding_train = all_embeddings[train_idxs][:, 1:, :]
embedding_val = all_embeddings[val_idxs][:, 1:, :]
embedding_test = all_embeddings[test_idxs][:, 1:, :]

In [17]:
%load_ext autoreload
%autoreload 2
from ml_utils.grid_search import GridSearch
from ml_utils.PCA_discriminator import PCADiscriminator
from ml_utils.metrics import auroc,get_best_split_from_scores

Grid Search for Best 'n_components' & 'layer'

In [18]:
def evaluator(n_components,i_layer,X,y):
    discriminator=PCADiscriminator(n_components,X[:,i_layer,:])
    scores=discriminator.get_score()
    return auroc(scores,y)

In [19]:
# graid search for best hyper-parameters on validation set
grid={
    'n_components':range(1,12),
    'layer':range(embedding_val.shape[1])
}
grid_search=GridSearch(evaluator,grid,embedding_val,hall_label_val)
best_paras=grid_search.search()

Grid Searching for best n_components,layer


  0%|          | 0/11 [00:00<?, ?it/s]

found best n_components:1,layer:0 Score: 0.518805664680733
found best n_components:1,layer:2 Score: 0.5198983618215088


found best n_components:1,layer:3 Score: 0.5295938808960117
found best n_components:1,layer:4 Score: 0.5318313083747431
found best n_components:1,layer:5 Score: 0.5343809350365534
found best n_components:1,layer:7 Score: 0.5852303769805136
found best n_components:1,layer:10 Score: 0.6119103988344564
found best n_components:1,layer:13 Score: 0.6683404011759504
found best n_components:1,layer:14 Score: 0.8181960090537762
found best n_components:1,layer:15 Score: 0.8391393709186461


100%|██████████| 11/11 [00:23<00:00,  2.17s/it]


In [20]:
best_paras

{'n_components': 1, 'layer': 15, 'best_score': np.float64(0.8391393709186461)}

In [21]:
best_layer=best_paras['layer']

View Results on Training set and Test set

In [22]:
discriminator=PCADiscriminator(best_paras['n_components'],embedding_val[:,best_layer,:])
best_split=discriminator.get_best_split(hall_label_val)

In [23]:
discriminator=PCADiscriminator(best_paras['n_components'],embedding_train[:,best_layer,:])
scores=discriminator.get_score()
y_train=(scores>best_split)
print(f'train set acc: {accuracy_score(hall_label_wild,y_train)} AUROC:{roc_auc_score(hall_label_wild,scores)}')

train set acc: 0.8755333333333334 AUROC:0.8524999157709654


In [24]:
discriminator=PCADiscriminator(best_paras['n_components'],embedding_test[:,best_layer,:])
scores=discriminator.get_score()
preds=(scores>best_split)
print(f'test set acc: {accuracy_score(hall_label_test,preds)} AUROC:{roc_auc_score(hall_label_test,scores)}')

test set acc: 0.865 AUROC:0.8602835971007319


Train Linear Prob to Get Better Results

In [25]:
X_train=embedding_train[:, best_layer,:]
y_train=y_train

X_test=embedding_test[:, best_layer,:]
y_test=hall_label_test

In [26]:
best_acc, final_acc, (
            clf, best_state, best_preds, preds, labels_val), losses_train = get_linear_acc(
            X_train,
            y_train,
           X_train,
            y_train,
            2, epochs=50,
            print_ret=True,
            batch_size=512,
            cosine=True,
            nonlinear=True,
            learning_rate=0.05,
            weight_decay=0.0003)

In [27]:
clf.eval()
output = clf(torch.from_numpy(X_test).to(torch.float32).cuda())
scores_test = torch.sigmoid(output).cpu().data.numpy()
pred_test=scores_test>0.5
print(f'test set acc: {accuracy_score(y_test,pred_test)} AUROC:{roc_auc_score(y_test,scores_test)}')

test set acc: 0.87625 AUROC:0.7811235095414049
